# Neural Networks in NLP HW 4

## Дарья Родионова

In [1]:
! pip install -q -U watermark

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.3.6 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 2.1.3 which is incompatible.
keyring 23.4.0 requires importlib-metadata>=3.6, but you have importlib-metadata 2.1.3 which is incompatible.
ibis-framework 2.1.1 requires importlib-metadata<5,>=4; python_version < "3.8", but you have importlib-metadata 2.1.3 which is incompatible.
gym 0.23.1 requires importlib-metadata>=4.10.0; python_version < "3.10", but you have importlib-metadata 2.1.3 which is incompatible.
arviz 0.11.4 requires typing-extensions<4,>=3.7.4.3, but you have typing-extensions 4.1.1 which is incompatible.


In [2]:
! pip install transformers

In [3]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

Python implementation: CPython
Python version       : 3.7.12
IPython version      : 7.30.1

numpy       : 1.20.3
pandas      : 1.3.5
torch       : 1.9.1
transformers: 4.16.2



Скачиваем нужные модули.

In [4]:
import transformers
from transformers import BertModel, AutoModel,AutoTokenizer, BertTokenizer, PreTrainedTokenizerFast, AdamW, get_linear_schedule_with_warmup
import torch.nn.functional as F

import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import Dataset, DataLoader

In [5]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 8, 6

In [6]:
! pip install datasets

### Downloading Data

Загружаем данные и делим их сразу на train/val/test.

In [7]:
from datasets import load_dataset

imdb_datasets = load_dataset('imdb', 
                             split={'train': 'train[:2000]+train[-2000:]', 
                                    'test': 'test[:1000]+test[-1000:]', 
                                    'validation': 'test[1000:2000]+test[-2000:-1000]'})

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
imdb_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

Решила выбрать модель bert-small, так как она маленькая и быстрая. 

In [9]:
MODEL_NAME = 'prajjwal1/bert-small'

In [10]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/286 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

### Preprocessing

Токенизируем данные. 

In [11]:
def preprocess(texts, tokenizer):
    result = tokenizer(texts['text'], 
                       max_length=512, 
                       padding='max_length', 
                       truncation=True)
    
    result['label'] = texts['label']
    
    return result

In [12]:
from functools import partial

tokenized_datasets = imdb_datasets.map(
    partial(preprocess, tokenizer=tokenizer),
    batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [13]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

Фукнция для подсчёта метрик. 

In [14]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

Задаём нужные параметры дляя обучения моделей.

In [15]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1'
)

In [16]:
data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)

In [17]:
from torch.nn import CrossEntropyLoss

### 1. SentimentClassifier

Немного видоизменённый класс SentimentClassifer с семинара. 

In [18]:
class SentimentClassifier(nn.Module):
    
    def __init__(self, n_classes):
        super().__init__()
        self.bert = AutoModel.from_pretrained(MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
        self.n_classes = n_classes
  
    def forward(self, input_ids, attention_mask, labels=None):
        last_hidden_state, pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False)
       
        logits = self.out(self.drop(pooled_output))

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits.view(-1, self.n_classes), labels.view(-1))

        output = (logits,)
        return ((loss,) + output) if loss is not None else output

In [19]:
model = SentimentClassifier(2)
model = model.to('cuda')

Downloading:   0%|          | 0.00/111M [00:00<?, ?B/s]

Some weights of the model checkpoint at prajjwal1/bert-small were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Обучаем.

In [20]:
trainer = Trainer(
    model=model,                        
    args=training_args,                  
    train_dataset=tokenized_datasets['train'],         
    eval_dataset=tokenized_datasets['validation'],           
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer_result = trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The following columns in the training set  don't have a corresponding argument in `SentimentClassifier.forward` and have been ignored: text, token_type_ids.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4000
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1000


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.323100,0.334828,0.871500,0.869211,0.884974,0.854000
2,0.254000,0.354860,0.896500,0.896861,0.893744,0.900000


The following columns in the evaluation set  don't have a corresponding argument in `SentimentClassifier.forward` and have been ignored: text, token_type_ids.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `SentimentClassifier.forward` and have been ignored: text, token_type_ids.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special

In [21]:
trainer_result.metrics

{'train_runtime': 260.9232,
 'train_samples_per_second': 30.66,
 'train_steps_per_second': 3.833,
 'total_flos': 0.0,
 'train_loss': 0.40278061163425444,
 'epoch': 2.0}

Проверяем на данных из тестовой выборки.

In [22]:
tester_result = trainer.predict(test_dataset=tokenized_datasets['test'])

The following columns in the test set  don't have a corresponding argument in `SentimentClassifier.forward` and have been ignored: text, token_type_ids.
***** Running Prediction *****
  Num examples = 2000
  Batch size = 8


In [23]:
tester_result.metrics

{'test_loss': 0.44090044498443604,
 'test_accuracy': 0.8775,
 'test_f1': 0.8787728847105393,
 'test_precision': 0.8697355533790402,
 'test_recall': 0.888,
 'test_runtime': 10.8429,
 'test_samples_per_second': 184.453,
 'test_steps_per_second': 23.057}

### 2. SentimentClassifier with CLS

Добавляем эмбеддинг [CLS] токенов с последнего слоя.

In [24]:
class SentimentClassifierCLS(nn.Module):
    
    def __init__(self, n_classes):
        super().__init__()
        self.bert = AutoModel.from_pretrained(MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size*2, n_classes)
        self.n_classes = n_classes
  
    def forward(self, input_ids, attention_mask, labels=None):
        last_hidden_state, pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False)
       

        cls = last_hidden_state[:,0,:] # [CLS]
        stacked_layers = torch.hstack([cls, pooled_output])

        logits = self.out(self.drop(stacked_layers))

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits.view(-1, self.n_classes), labels.view(-1))

        output = (logits,)
        
        return ((loss,) + output) if loss is not None else output

In [25]:
model = SentimentClassifierCLS(2)
model = model.to('cpu')

loading configuration file https://huggingface.co/prajjwal1/bert-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ac031779e2b4dd1d9da1e39c9d6a29fd45deea195eb3703a701d9c77f60abb4e.1257bb8f1f585038e86954d2560e36ca5c2dd98a8cde30fd22468940c911b672
Model config BertConfig {
  "_name_or_path": "prajjwal1/bert-small",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/prajjwal1/bert-small/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transfor

Обучаем. 

In [26]:
trainer = Trainer(
    model=model,                        
    args=training_args,                  
    train_dataset=tokenized_datasets['train'],         
    eval_dataset=tokenized_datasets['validation'],           
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer_result_cls = trainer.train()

The following columns in the training set  don't have a corresponding argument in `SentimentClassifierCLS.forward` and have been ignored: text, token_type_ids.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4000
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.265700,0.339454,0.871500,0.870919,0.874874,0.867000
2,0.169400,0.382169,0.887500,0.886764,0.892604,0.881000


The following columns in the evaluation set  don't have a corresponding argument in `SentimentClassifierCLS.forward` and have been ignored: text, token_type_ids.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `SentimentClassifierCLS.forward` and have been ignored: text, token_type_ids.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/s

In [27]:
trainer_result_cls.metrics

{'train_runtime': 139.7181,
 'train_samples_per_second': 57.258,
 'train_steps_per_second': 7.157,
 'total_flos': 0.0,
 'train_loss': 0.4256142890453339,
 'epoch': 2.0}

Проверяем на данных из тестовой выборки.

In [28]:
tester_result_cls = trainer.predict(test_dataset=tokenized_datasets['test'])

The following columns in the test set  don't have a corresponding argument in `SentimentClassifierCLS.forward` and have been ignored: text, token_type_ids.
***** Running Prediction *****
  Num examples = 2000
  Batch size = 8


In [29]:
tester_result_cls.metrics

{'test_loss': 0.4390011727809906,
 'test_accuracy': 0.874,
 'test_f1': 0.8738738738738739,
 'test_precision': 0.874749498997996,
 'test_recall': 0.873,
 'test_runtime': 9.2433,
 'test_samples_per_second': 216.372,
 'test_steps_per_second': 27.047}

### 3. BertForSequenceClassification

Всё просто, скачиваем предобученный БЕРТ для классификации, аналогично обучаем и тестируем. 

In [30]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(MODEL_NAME)
model = model.to('cpu')

loading configuration file https://huggingface.co/prajjwal1/bert-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ac031779e2b4dd1d9da1e39c9d6a29fd45deea195eb3703a701d9c77f60abb4e.1257bb8f1f585038e86954d2560e36ca5c2dd98a8cde30fd22468940c911b672
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/prajjwal1/bert-small/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/facfdb1638fdec899406e0efd5c2c43ae4bbaf

In [31]:
trainer = Trainer(
    model=model,                        
    args=training_args,                  
    train_dataset=tokenized_datasets['train'],         
    eval_dataset=tokenized_datasets['validation'],           
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer_result_pre = trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4000
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.282800,0.301622,0.879000,0.877654,0.887526,0.868000
2,0.129500,0.388723,0.890000,0.891304,0.880859,0.902000


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./result

In [32]:
trainer_result_pre.metrics

{'train_runtime': 130.8263,
 'train_samples_per_second': 61.15,
 'train_steps_per_second': 7.644,
 'total_flos': 316397371392000.0,
 'train_loss': 0.3838183356523514,
 'epoch': 2.0}

In [33]:
tester_results_pre = trainer.predict(test_dataset=tokenized_datasets['test'])

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 2000
  Batch size = 8


In [34]:
tester_results_pre.metrics

{'test_loss': 0.46258434653282166,
 'test_accuracy': 0.874,
 'test_f1': 0.8771929824561403,
 'test_precision': 0.8555133079847909,
 'test_recall': 0.9,
 'test_runtime': 9.894,
 'test_samples_per_second': 202.142,
 'test_steps_per_second': 25.268}

### 4. SentimentClassifier with CLS tokens from all layers

Агрегируем [CLS] токены для всех слоёв и находим среднее.

In [35]:
class SentimentClassifierPooledCLS(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.n_classes = n_classes
        self.bert = AutoModel.from_pretrained(MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size*2, n_classes)
  
    def forward(self, input_ids, attention_mask, labels=None):
        _, pooled_output, hidden_states = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False,
            output_hidden_states=True)
       
        hidden_states = torch.stack(hidden_states)
        hidden_cls = hidden_states[:,:,1,:]
        hidden_cls = hidden_cls.mean(axis=0)

        stacked_layers = torch.hstack([hidden_cls, pooled_output])

        logits = self.out(self.drop(stacked_layers))

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits.view(-1, self.n_classes), labels.view(-1))

        output = (logits,)
        return ((loss,) + output) if loss is not None else output

In [36]:
model = SentimentClassifierPooledCLS(2)
model = model.to('cpu')

loading configuration file https://huggingface.co/prajjwal1/bert-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ac031779e2b4dd1d9da1e39c9d6a29fd45deea195eb3703a701d9c77f60abb4e.1257bb8f1f585038e86954d2560e36ca5c2dd98a8cde30fd22468940c911b672
Model config BertConfig {
  "_name_or_path": "prajjwal1/bert-small",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 8,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/prajjwal1/bert-small/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transfor

In [37]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer_results_pooled = trainer.train()

The following columns in the training set  don't have a corresponding argument in `SentimentClassifierPooledCLS.forward` and have been ignored: text, token_type_ids.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4000
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.261200,0.346265,0.862000,0.858896,0.878661,0.840000
2,0.234400,0.378114,0.887000,0.885859,0.894898,0.877000


The following columns in the evaluation set  don't have a corresponding argument in `SentimentClassifierPooledCLS.forward` and have been ignored: text, token_type_ids.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `SentimentClassifierPooledCLS.forward` and have been ignored: text, token_type_ids.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/check

In [38]:
trainer_results_pooled.metrics

{'train_runtime': 127.0275,
 'train_samples_per_second': 62.978,
 'train_steps_per_second': 7.872,
 'total_flos': 0.0,
 'train_loss': 0.43764497292041776,
 'epoch': 2.0}

In [39]:
tester_results_pooled = trainer.predict(test_dataset=tokenized_datasets['test'])

The following columns in the test set  don't have a corresponding argument in `SentimentClassifierPooledCLS.forward` and have been ignored: text, token_type_ids.
***** Running Prediction *****
  Num examples = 2000
  Batch size = 8


In [40]:
tester_results_pooled.metrics

{'test_loss': 0.4512108564376831,
 'test_accuracy': 0.8695,
 'test_f1': 0.869172932330827,
 'test_precision': 0.871356783919598,
 'test_recall': 0.867,
 'test_runtime': 9.2646,
 'test_samples_per_second': 215.874,
 'test_steps_per_second': 26.984}

### Model Comparison

1. <b>SentimentClassifier: 0.8787</b>
2. SentimentClassifier with CSL: 0.8738
3. BertForSequenceClassification: 0.8771
4. SentimentClassifier with CLS tokens from all layers: 0.8691

В целом все модели показали достаточно высокие результаты. Интересно, что лучшие результаты показала самая первая модель.

# GooglePlay Reviews

Посмотрим, как работает последняя модель на отзывах из GooglePlay.

In [41]:
! pip install gdown

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14775 sha256=4e253c3d8fffdd7fdab50fd7ba8a99bce2c29c3dbc6df4c63ae8f193f3a4aa79
  Stored in directory: /root/.cache/pip/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown


In [44]:
! gdown --id 1zdmewp7ayS4js4VtrJEHzAheSW-5NBZv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/opt/conda/lib/python3.7/site-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1zdmewp7ayS4js4VtrJEHzAheSW-5NBZv
To: /kaggle/working/reviews.csv
100%|██████████████████████████████████████| 7.17M/7.17M [00:00<00:00, 71.7MB/s]


Посмотрим на датасет с отзывами. 

In [45]:
df = pd.read_csv('reviews.csv')
df

,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sortOrder,appId
0,Andrew Thomas,https://lh3.googleusercontent.com/a-/AOh14GiHd...,Update: After getting a response from the deve...,1,21,4.17.0.3,2020-04-05 22:25:57,"According to our TOS, and the term you have ag...",2020-04-05 15:10:24,most_relevant,com.anydo
1,Craig Haines,https://lh3.googleusercontent.com/-hoe0kwSJgPQ...,Used it for a fair amount of time without any ...,1,11,4.17.0.3,2020-04-04 13:40:01,It sounds like you logged in with a different ...,2020-04-05 15:11:35,most_relevant,com.anydo
2,steven adkins,https://lh3.googleusercontent.com/a-/AOh14GiXw...,Your app sucks now!!!!! Used to be good but no...,1,17,4.17.0.3,2020-04-01 16:18:13,This sounds odd! We are not aware of any issue...,2020-04-02 16:05:56,most_relevant,com.anydo
3,Lars Panzerbjørn,https://lh3.googleusercontent.com/a-/AOh14Gg-h...,"It seems OK, but very basic. Recurring tasks n...",1,192,4.17.0.2,2020-03-12 08:17:34,We do offer this option as part of the Advance...,2020-03-15 06:20:13,most_relevant,com.anydo
4,Scott Prewitt,https://lh3.googleusercontent.com/-K-X1-YsVd6U...,Absolutely worthless. This app runs a prohibit...,1,42,4.17.0.2,2020-03-14 17:41:01,We're sorry you feel this way! 90% of the app ...,2020-03-15 23:45:51,most_relevant,com.anydo
...,...,...,...,...,...,...,...,...,...,...,...
15741,Tammy Kay,https://lh3.googleusercontent.com/a-/AOh14GhYP...,I believe that this is by far the best app wit...,5,0,NaN,2018-02-17 06:09:03,NaN,NaN,newest,com.appxy.planner
15742,Ysm Johan,https://lh3.googleusercontent.com/a-/AOh14Ggmd...,It sometimes crashes a lot!!,5,0,4.3.7,2018-02-15 10:45:22,NaN,NaN,newest,com.appxy.planner
15743,casey dearden,https://lh3.googleusercontent.com/a-/AOh14Gg2U...,Works well for what I need,5,0,4.3.7,2018-02-09 18:40:37,NaN,NaN,newest,com.appxy.planner
15744,Jerry G Tamate,https://lh3.googleusercontent.com/a-/AOh14GiTP...,Love it.,5,0,NaN,2018-02-06 12:36:17,NaN,NaN,newest,com.appxy.planner


In [47]:
negative_review = df[df.score<=2]

In [48]:
negative_review.content[3]

"It seems OK, but very basic. Recurring tasks need some work to be actually useful. For example, it would be nice to be able to set a task to be recurring on the first of every month, without only being able to set that up on the first of the month. Edit; I also just noticed that there is no dark theme. Both may be available as paid for options, but I'll never know, since they are basic options and without them, I have no reason to try this app, and thus will never pay for actual premium options."

In [49]:
negative = '''It seems OK, but very basic. Recurring tasks need 
some work to be actually useful. 
For example, it would be nice to be able to set a task to be recurring on the first of every month, 
without only being able to set that up on the first of the month. 
Edit; I also just noticed that there is no dark theme. 
Both may be available as paid for options, but I'll never know, 
since they are basic options and without them, 
I have no reason to try this app, and thus will never pay for actual premium options.'''

encoding = tokenizer.encode_plus(
  negative,
  max_length=512,
  add_special_tokens=True, 
  return_token_type_ids=True,
  padding='max_length',
  return_attention_mask=True,
  return_tensors='pt',  
  truncation=True
).to('cuda')

In [50]:
model(
  input_ids=encoding['input_ids'], 
  attention_mask=encoding['attention_mask'])[0].cpu().detach().numpy().argmax()

0

Отрицательная полярность для отрицательного отзыва. 

In [51]:
neutral_review = df[df.score==3]

In [52]:
neutral_review.content[400]

"I love the concept of the app I'm just not digging the functionality. Trying to set a custom date and time is a complete pain in the you know what. Every time I select pm it's stays at am. I have to do it multiple times before it catches. Same issue with the clock. I try to rotate the dial to select an hour and it hangs and throws me over to minutes. I go back to hours and it does the same thing. I finally just gave up. Back to the old string around the finger. ☹️"

In [53]:
neutral = '''I love the concept of the app I'm just not digging the functionality. 
Trying to set a custom date and time is a complete pain in the you know what. 
Every time I select pm it's stays at am. I have to do it multiple 
times before it catches. Same issue with the clock. 
I try to rotate the dial to select an hour and it hangs and 
throws me over to minutes. I go back to hours and it does the same thing. 
I finally just gave up. Back to the old string around the finger. ☹️'''

encoding = tokenizer.encode_plus(
  neutral,
  max_length=512,
  add_special_tokens=True, 
  return_token_type_ids=True,
  padding='max_length',
  return_attention_mask=True,
  return_tensors='pt',  
  truncation=True
).to('cuda')

In [54]:
model(
  input_ids=encoding['input_ids'], 
  attention_mask=encoding['attention_mask'])[0].cpu().detach().numpy().argmax()

1

Положительная полярность для нейтральной оценки.

In [55]:
positive_review = df[df.score>3]

In [56]:
positive_review.content[15741]

"I believe that this is by far the best app with helping ppl keep track of a lot of daily task and reminders. It's easy to function understand & it syncs with my Google calendar which is an A+ in my book."

In [57]:
positive = '''I believe that this is by far the best app with helping 
ppl keep track of a lot of daily task and reminders. 
It's easy to function understand & it syncs with my Google calendar which is an A+ in my book.'''

encoding = tokenizer.encode_plus(
  positive,
  max_length=512,
  add_special_tokens=True, 
  return_token_type_ids=True,
  padding='max_length',
  return_attention_mask=True,
  return_tensors='pt',  
  truncation=True
).to('cuda')

In [58]:
model(
  input_ids=encoding['input_ids'], 
  attention_mask=encoding['attention_mask'])[0].cpu().detach().numpy().argmax()


1

Положительная полярность для положительного отзыва. 

В целом модель работает корректно, пусть и отзывы рассчитаны на 3 класса.